In [1]:
!pip install -r /kaggle/input/depthanythingv2/transformers/default/1/Depth-Anything-V2/requirements.txt

In [2]:
import os, gc, cv2, sys, torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from PIL import Image
from tqdm import tqdm
import torchvision.transforms as T

sys.path.append("/kaggle/input/depthanythingv2/transformers/default/1/Depth-Anything-V2")

from depth_anything_v2.dpt import DepthAnythingV2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DepthAnythingV2(encoder='vitl', features=256, out_channels=[256, 512, 1024, 1024])
model.load_state_dict(torch.load(
    '/kaggle/input/depthanythingv2/transformers/default/1/Depth-Anything-V2/checkpoints/depth_anything_v2_vitl.pth',
    map_location=device
))
model.to(device)
model.eval();

# ========== 路径配置 ==========
base_dir = "/kaggle/input/kitti-dataset/depth_selection/val_selection_cropped"
img_dir = os.path.join(base_dir, "image")
gt_dir = os.path.join(base_dir, "groundtruth_depth")
K_dir = os.path.join(base_dir, "intrinsics")

save_dir = "/kaggle/working/"
os.makedirs(save_dir, exist_ok=True)
vis_dir = os.path.join(save_dir, "visualizations")
os.makedirs(vis_dir, exist_ok=True)
pres_dir = os.path.join("/kaggle/working", "pres")
os.makedirs(pres_dir, exist_ok=True)

# ========== 可视化 ==========
def save_vis(rgb, gt, pred, mask, save_path):
    plt.figure(figsize=(15, 5))
    valid = mask & (gt > 0) & np.isfinite(gt) & np.isfinite(pred)
    all_depths = np.concatenate([gt[valid], pred[valid]])
    vmin = np.percentile(all_depths, 2)
    vmax = np.percentile(all_depths, 98)

    plt.subplot(1, 3, 1)
    plt.imshow(rgb)
    plt.title("RGB")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(gt, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pred, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# ========== 评估函数 ==========
def compute_abs_rel(pred, gt, mask=None):
    # 支持 mask=None
    if mask is None:
        valid = (gt > 0)
    else:
        valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    if len(gt) == 0:
        return np.nan
    return np.mean(np.abs(pred - gt) / gt)

def compute_delta1(pred, gt, mask):
    valid = mask & (gt > 1e-6) & (pred > 1e-6)
    pred, gt = pred[valid], gt[valid]
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    pred, gt = pred[mask], gt[mask]
    return np.sum(gt * pred) / np.sum(pred ** 2)

# ========== 图像预处理 ==========
transform = T.Compose([
    T.Resize((518, 518)),  # Depth Anything 默认输入分辨率
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ========== 主流程 ==========
results = []
img_files = sorted([f for f in os.listdir(img_dir) if f.endswith(".png")])

for img_file in tqdm(img_files, desc="Evaluating"):
    try:
        # 构建文件路径
        rgb_path = os.path.join(img_dir, img_file)
        gt_file = re.sub("image", "groundtruth_depth", img_file, count=1)
        gt_path = os.path.join(gt_dir, gt_file)
        K_path = os.path.join(K_dir, img_file.replace(".png", ".txt"))

        # 加载图像
        rgb = cv2.imread(rgb_path)
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        rgb_pil = Image.fromarray(rgb)

        # GT 深度图
        gt_png = cv2.imread(gt_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
        gt_depth = gt_png / 256.0
        mask = gt_depth > 0

        # 图像预处理
        input_tensor = transform(rgb_pil).unsqueeze(0).to(device)

        # 推理
        with torch.no_grad():
            pred = model(input_tensor)
        pred_depth = pred.squeeze().cpu().numpy()

        # Resize 到 GT 尺寸
        if pred_depth.shape != gt_depth.shape:
            pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

        # 缩放匹配
        scale = scale_match(pred_depth, gt_depth, mask)
        pred_aligned = pred_depth * scale

        # 评估指标
        absrel = compute_abs_rel(pred_aligned, gt_depth, mask)
        delta1 = compute_delta1(pred_aligned, gt_depth, mask)
        results.append([img_file, absrel, delta1])
        print(f"[{img_file}] AbsRel: {absrel:.4f}, δ1: {delta1 * 100:.2f}%")

        # 保存可视化图
        vis_path = os.path.join(vis_dir, img_file.replace(".png", "_vis.png"))
        save_vis(rgb, gt_depth, pred_aligned, mask, vis_path)

        # 保存深度图
        np.save(os.path.join(pres_dir, img_file.replace(".png", "_pred.npy")), pred_aligned)

        del rgb, gt_depth, pred, pred_aligned, input_tensor
        torch.cuda.empty_cache()
        gc.collect()

    except Exception as e:
        print(f"跳过 {img_file}，错误：{e}")
        continue

# 保存评估结果（均值)
df = pd.DataFrame(results, columns=["image", "AbsRel", "Delta1"])

mean_absrel = df["AbsRel"].mean()
mean_delta1 = df["Delta1"].mean()
mean_row = pd.DataFrame([["mean", mean_absrel, mean_delta1]], columns=["image", "AbsRel", "Delta1"])

df = pd.concat([df, mean_row], ignore_index=True)
df.to_csv(os.path.join(save_dir, "eval_results.csv"), index=False)
print(f"\n所有评估结果保存至：{os.path.join(save_dir, 'eval_results.csv')}")

#dabao
import shutil
# 压缩 visualizations 文件夹
vis_zip_path = os.path.join(save_dir, "visualizations.zip")
shutil.make_archive(base_name=vis_zip_path.replace(".zip", ""), format='zip', root_dir=vis_dir)
print(f"visualizations 已打包至: {vis_zip_path}")

# 压缩 pres 文件夹
pres_dir = os.path.join(save_dir, "pres")  # pres 文件夹路径
pres_zip_path = os.path.join(save_dir, "pres.zip")
shutil.make_archive(base_name=pres_zip_path.replace(".zip", ""), format='zip', root_dir=pres_dir)
print(f"pres 已打包至: {pres_zip_path}")

Evaluating:   0%|          | 0/1000 [00:00<?, ?it/s]

[2011_09_26_drive_0002_sync_image_0000000005_image_02.png] AbsRel: 0.6783, δ1: 14.47%


Evaluating:   0%|          | 1/1000 [00:01<25:52,  1.55s/it]

[2011_09_26_drive_0002_sync_image_0000000008_image_03.png] AbsRel: 0.6600, δ1: 16.94%


Evaluating:   0%|          | 2/1000 [00:02<21:18,  1.28s/it]

...

[2011_10_03_drive_0047_sync_image_0000000785_image_02.png] AbsRel: 0.7332, δ1: 10.02%


Evaluating: 100%|█████████▉| 999/1000 [52:39<00:05,  5.23s/it]

[2011_10_03_drive_0047_sync_image_0000000791_image_03.png] AbsRel: 0.7629, δ1: 9.65%


Evaluating: 100%|██████████| 1000/1000 [52:44<00:00,  3.16s/it]


✅ 所有评估结果保存至：/kaggle/working/eval_results.csv
